In [ ]:
import h5py
from torchvision import datasets, transforms
import torch
from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Select device which is available

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

# From CSV to HDF5 File and Shuffle

In [ ]:
def CreateDatasetHDF5(pathdir):
    with h5py.File('pointcloud2_hdf5.h5','w') as hdf:
        defect_group = hdf.create_group('Defect')
        fine_group = hdf.create_group('Fine')
        
        groups = [defect_group,fine_group]
        folders = os.listdir(f'{pathdir}')
        
        zipping = zip(groups,folders)
        
        print(f'Converting cvs to hdf5 as: "pointcloud2_hdf5.h5".')
        for group, folder in zipping:
            print(f'Converting {folder} cvs...')
            for idx,pointcloud in enumerate(tqdm(os.listdir(f'{pathdir}/{folder}'))):
                
                array = np.genfromtxt(f'{pathdir}/{folder}/{pointcloud}',delimiter=',')
                
                #print(len(array))
                if len(array) < POINTCLOUD_SIZE:
                    print("Yup")
                
                array = np.delete(array,slice(POINTCLOUD_SIZE,len(array)),0)
                
                group.create_dataset(f'pc{idx}',data=array)

In [ ]:
def shuffle():
    with h5py.File('pointcloud2_hdf5.h5','r') as hdf:
        combined_pointcloud_list = []
        combined_label_list = []

        for index, group in enumerate(list(hdf.keys())):
            pointcloud_list = list(hdf.get(f'{group}'))

            for pointcloud in pointcloud_list:
                pointcloud = hdf.get(f'Defect/{pointcloud}')
                pointcloud = np.array(pointcloud)
                
                combined_pointcloud_list.append(pointcloud)
                combined_label_list.append(np.eye(2)[index])

        temp_zip = list(zip(combined_pointcloud_list,combined_label_list))
        random.shuffle(temp_zip)
        pointclouds, labels = zip(*temp_zip)
        
        return pointclouds, labels

In [ ]:
def combine_and_shuffle(test_ratio = 0.15):
    pointclouds, labels = shuffle()
    with h5py.File('shuffled_pointcloud2_hdf5.h5','w') as hdf:
        size = int(len(pointclouds)*test_ratio)
        
        #for group in list(hdf.keys()): #Training and Testing
        #    group = hdf.get(f'{group}')
        #    pointcloud_group = group.create_group('PointClouds')
        #    label_group = group.create_group('Labels')
        
        testing_group = hdf.create_group('Testing')
        pointcloud_group = testing_group.create_group('PointClouds')
        label_group = testing_group.create_group('Labels')
        for idx,(pointcloud,label) in enumerate(zip(pointclouds[-size:],labels[-size:])):
            pointcloud_group.create_dataset(f'pc{idx}',data=pointcloud)
            label_group.create_dataset(f'pc{idx}',data=label)   
            
            
        training_group = hdf.create_group('Training')
        pointcloud_group = training_group.create_group('PointClouds')
        label_group = training_group.create_group('Labels')
        for idx,(pointcloud,label) in enumerate(zip(pointclouds[:-size],labels[:-size])):
            pointcloud_group.create_dataset(f'pc{idx}',data=pointcloud)
            label_group.create_dataset(f'pc{idx}',data=label)
            

In [ ]:
POINTCLOUD_SIZE = 9500

CreateDatasetHDF5('../input/pointcloudv2/PointCloudV2')

combine_and_shuffle(test_ratio = 0.2)
with h5py.File('shuffled_pointcloud2_hdf5.h5','r') as hdf:
    
    for key in hdf.keys():
        print(key)
        print(list(hdf.get(f'{key}/')))
        print(hdf.get(f'{key}/Labels'))
        print(hdf.get(f'{key}/PointClouds'))
        print('\n')

# PointNet Architecture

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Tnet(nn.Module):
    def __init__(self,k):
        super().__init__()
        self.k = k
        
        self.conv1 = nn.Conv1d(in_channels = k,out_channels = 64,kernel_size=1) #k instead of 3
        self.conv2 = nn.Conv1d(in_channels = 64,out_channels = 128,kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels = 128,out_channels = 1024,kernel_size=1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k*k)
        
    
    def forward(self,x):
        point_cloud_size = len(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x = torch.max(x,2,keepdim=True)[0]
        x = x.view(-1,1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        
        #if self.k == 3:
        #    iden = torch.from_numpy(np.eye(self.k).flatten().astype(np.float32)).view(1,self.k*self.k).repeat(point_cloud_size,1).to(device)
        #elif self.k == 64:
        iden = torch.from_numpy(np.eye(self.k).flatten().astype(np.float32)).view(1,self.k*self.k).repeat(point_cloud_size,1).to(device)
        
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x
        

In [ ]:
class InputTransform(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.tnet = Tnet(3).to(device)
        
    def forward(self,x):
        input_trans = self.tnet(x).to(device)
        x = x.transpose(2,1) #??
        x = torch.bmm(x,input_trans)
        x = x.transpose(2,1) #??
        return x

In [ ]:
class FeatureTransform(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.tnet = Tnet(64)
        
    def forward(self,x):
        feature_transform = self.tnet(x)
        x = x.transpose(2,1) #??
        x = torch.bmm(x,feature_transform)
        x = x.transpose(2,1) #??
        return x

In [ ]:
class Classification(nn.Module):
    def __init__(self,k=2):
        super().__init__()
    
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k)
        
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.7)
    
    def forward(self,x):
        x = self.dropout(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout(F.relu(self.bn2(self.fc2(x))))
        x = F.softmax(self.fc3(x),dim=1)
        return x

![](https://stanford.edu/~rqi/pointnet/images/pointnet.jpg)

In [ ]:
class PointNetCls(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input_transform = InputTransform()
        self.feature_transform = FeatureTransform()
        
        self.conv1 = nn.Conv1d(3,64,1) #???? really value? Arcitecture says (64/64)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1) 
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        
        self.classification = Classification()
        
    def forward(self,x):
        n_points = x.shape[0] #n x 3
        
        x = self.input_transform(x) #Input Transform
        
        x = F.relu(self.bn1(self.conv1(x))) #MLP Shared(64,64)             ???? really value? Arcitecture says (64/64)
        
        #n x 64
        
        x = self.feature_transform(x) #Feature Transform
        
        point_feature = x #n x 64 (Point Features for segmentation (Maybe use later))
        
        x = F.relu(self.bn2(self.conv2(x))) #MLP 64 --> 128
        x = self.bn3(self.conv3(x)) #MLP 128 --> 1024
        
        x = torch.max(x, 2, keepdim=True)[0]
            
        x = x.view(-1,1024) #Global Feature (Missing Link to segment)
        
        x = self.classification(x) #MLP (512,256,k) + output scores // Classification
        
        return x

In [ ]:
def number_of_pointclouds(group):
    with h5py.File('shuffled_pointcloud2_hdf5.h5','r') as hdf:
        n_pointclouds = len(list(hdf.get(f'{group}/PointClouds')))
        return n_pointclouds

In [ ]:
def get_batch(index,batch_size):
    pointcloud_list = []
    label_list = []
    
    with h5py.File('shuffled_pointcloud2_hdf5.h5','r') as hdf:

        for pointcloud in list(hdf.get('Training/PointClouds'))[index:index+batch_size]:
            array = np.array(hdf.get(f'Training/PointClouds/{pointcloud}'))
            pointcloud_list.append(array)

        for label in list(hdf.get('Training/Labels'))[index:index+batch_size]:
            array = np.array(hdf.get(f'Training/Labels/{label}'))
            label_list.append(array)
     
    pointcloud_tensors = torch.Tensor(pointcloud_list).view(-1,3,POINTCLOUD_SIZE).to(device)
    label_tensors = torch.Tensor(label_list).to(device)

    return pointcloud_tensors, label_tensors

# Training

In [ ]:
EPOCHS = 120
BATCH_SIZE = 32

def train(net):
    loss_list = []
    optimizer = torch.optim.Adam(net.parameters(),lr=0.001,betas=(0.9, 0.999)) #MOMENTUM 0.9
    loss_function = nn.MSELoss()
    
    for epoch in range(EPOCHS):
        for i in tqdm(range(0,number_of_pointclouds('Training'),BATCH_SIZE)):
            batch_X, batch_y = get_batch(i,BATCH_SIZE)
            #print(number_of_pointclouds('Training'))
            #print(batch_y.shape)
            #print(batch_y)
            net.zero_grad()
            outputs = net(batch_X)
            
            loss = loss_function(outputs,batch_y)
            loss.backward()
            loss_list.append(loss)
            
            optimizer.step()
    
        print(f'Epoch: {epoch}. Loss {loss}.')
    
    plt.plot(loss_list)
            
            

In [ ]:
pointnet_cls = PointNetCls().to(device)
train(pointnet_cls)

# Testing

In [ ]:
def get_testing_data():
    pointcloud_list = []
    label_list = []
    
    with h5py.File('shuffled_pointcloud2_hdf5.h5','r') as hdf:

        for pointcloud in list(hdf.get('Testing/PointClouds')):
            array = np.array(hdf.get(f'Testing/PointClouds/{pointcloud}'))
            pointcloud_list.append(array)

        for label in list(hdf.get('Testing/Labels')):
            array = np.array(hdf.get(f'Testing/Labels/{label}'))
            label_list.append(array)
     
    pointcloud_tensors = torch.Tensor(pointcloud_list).view(-1,3,POINTCLOUD_SIZE).to(device)
    label_tensors = torch.Tensor(label_list)

    return pointcloud_tensors, label_tensors

In [ ]:
def predict(net):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(number_of_pointclouds('Testing'))):
            test_X,test_y = get_testing_data()
            true_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1,3,POINTCLOUD_SIZE)).to(device)[0]
            predicted_class = torch.argmax(net_out)
            #print(f'Real Class: {true_class}')
            #print(f'Classification: {net_out}')
            #print(f'Predicted Class: {predicted_class} \n')
            

            if predicted_class == true_class:
                correct += 1
            total += 1
    print("Accuracy: ",round(correct/total,3))
    
predict(pointnet_cls)